In [ ]:
#导入包
import os
import cv2
import torch
import numpy as np
from torch.utils.data import DataLoader
from untils.dataset import SegmentationDataset
from Loss.BCELoss import BCELoss
from model.U_net.base import Model1
from train_val.base import train_and_validate
from untils.predict_utils import predict
from untils.evaluate_predictions import evaluate_predictions

In [ ]:
# 设置随机种子
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [ ]:
# 检查是否有可用的 CUDA 设备，如果有则使用 CUDA，否则使用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 创建训练集的数据加载器
train_loader = DataLoader(SegmentationDataset('crack2181/train'), batch_size=4, shuffle=True, num_workers=0)

# 创建验证集的数据加载器
val_loader = DataLoader(SegmentationDataset('crack2181/val'), batch_size=1, shuffle=False, num_workers=0)

# 定义模型参数
in_features = 3
out_features = 1
init_features = 32
#factor = 4
#dropout_rate = 0.1

# 创建 UNet 模型，并将模型移动到对应的设备上
model = Model1(in_features, out_features, init_features).to(device)
#model = Model2(in_features, out_features, init_features, factor).to(device)
#model = Model3(in_features, out_features, init_features, dropout_rate).to(device)
#model = Model4(in_features, out_features, init_features, factor, dropout_rate).to(device)

# 定义训练的相关参数
num_epochs = 3
train_on_gpu = True
loss_fn = BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
patience = 3

# 训练函数
train_and_validate(model, train_loader, val_loader, optimizer, loss_fn, num_epochs, patience, train_on_gpu)

# 预测函数
model.load_state_dict(torch.load('u-net.pt'))
model.eval()
input_dir = 'crack2181/test/images'
output_dir = 'crack2181/test/predict'
predict(model, input_dir, output_dir)

# 计算预测指标
prediction_dir = 'crack2181/test/predict'
true_dir = 'crack2181/test/labels'
evaluate_predictions(prediction_dir, true_dir)